# Replot threshold data for publication

I've decided not to invest time reanalysing this data with a GLMM-style thing. The paper doesn't warrant it. Instead, I am shooting for a minimal presentation of the results. To do so, I will use Saskia's *psignifit* fits to individual conditions. 

In this notebook I will just replot her fit data in a nicer format.

In [ ]:
import os
import numpy as np
import pandas as pd
import psyutils as pu
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize
from itertools import product

%matplotlib inline

sns.set_style('white')
sns.set_style('ticks')
sns.set_context('paper')

In [ ]:
# pal = sns.color_palette("colorblind", 2)
pal = sns.color_palette(['#282828', '#888888'])
sns.set_palette(pal)
sns.palplot(pal)

# Experiment 1

In [ ]:
top_dir = pu.files.project_directory('letter-distortion-detection')
fig_dir = os.path.join(top_dir, 'figures')
fname = os.path.join(top_dir, 'results', 'saskia_analysis', 'sensitivitydata', 'alldatasensexp1+2.csv')
dat = pd.read_csv(fname, sep='\t')
dat.info()

### Prepare data

In [ ]:
# remove leading spaces from variable names, rename cols:
def new_col(col):
    # rename columns containing leading space:
    col = col.rstrip()
    return col.lstrip()

dat.rename(columns=new_col, inplace=True)

dat.rename(columns={'subject': 'Observer',
                    'distortiontype': 'Distortion'}, 
           inplace=True)

dat.loc[dat['Distortion']==' Bex', 'Distortion'] = 'BPN'
dat.loc[dat['Distortion']==' RF', 'Distortion'] = 'RF'

dat.loc[:, 'log_freq'] = np.log(dat['freq'])


In [ ]:
dat.info()

In [ ]:
dat.head()

### Quantify peak tuning for BPN by fitting log Gaussian

In [ ]:
def inv_gauss_fun(x, pars):
    peak_x, peak_y, width, amp = pars
    gauss = np.exp(- ((x - peak_x)**2 / (2 * width**2) ))
    return amp * (1 - gauss) + peak_y


def error_fun(pars, x, y):
    yhat = inv_gauss_fun(x, pars)
    return ((y - yhat)**2).sum()  # minimise sum of squared errors
       

def expand_grid(data_dict):
    rows = product(*data_dict.values())
    return pd.DataFrame.from_records(rows, columns=data_dict.keys())

In [ ]:
x = np.linspace(0, 5, num=50)
pars = [2, -3, 1, 1]
y = inv_gauss_fun(x, pars)
plt.plot(x, y)

### Loop over subjects, condition

(I'm sure there's a way to do this with a groupby / apply operation)

In [ ]:
subdat = dat.loc[dat['Distortion']=='BPN', :].copy(deep=True)
param_dat = expand_grid({'Observer': np.unique(subdat['Observer']), 
                         'flanked': np.unique(subdat['flanked'])}) 

for s, c in product(np.unique(subdat['Observer']), np.unique(subdat['flanked'])):
    mask = (subdat['Observer']==s) & (subdat['flanked']==c)
    this_dat = subdat.loc[mask, :]
    print('Subject {}, condition {}'.format(s, c))
    res = minimize(error_fun, [1, 0.5, 2, 2], 
                   args=(this_dat['log_freq'], np.log(this_dat['threshold'])),
                   method='BFGS')
    print('converged = {}'.format(res.success))
    print(res.message)
    
    # save to frame:
    param_mask = (param_dat['Observer']==s) & (param_dat['flanked']==c)
    param_dat.loc[param_mask, 'peak_x'] = np.exp(res.x[0])
    param_dat.loc[param_mask, 'peak_y'] = np.exp(res.x[1])
    param_dat.loc[param_mask, 'width'] = res.x[2]
    param_dat.loc[param_mask, 'amp'] = res.x[3]
    
    # generate predictions for plotting:
    xhat = np.linspace(subdat['log_freq'].min(), subdat['log_freq'].max())
    yhat = inv_gauss_fun(xhat, res.x)
    pred_dat = expand_grid({'Observer': [s], 
                            'flanked': [c], 
                            'xhat': xhat})
    pred_dat['yhat'] = yhat
    subdat = subdat.append(pred_dat, ignore_index=True)
    

In [ ]:
subdat.info()

In [ ]:
def error_wrap(x, y, lo, hi, **kwargs):
    err = np.array([lo, hi])
    plt.errorbar(x, y, err, **kwargs)
    
    
def log_error_wrap(x, y, lo, hi, **kwargs):
    err = np.array([np.log(y) - np.log(y - lo), np.log(y + hi) - np.log(y)])
    plt.errorbar(x, np.log(y), err, **kwargs)
    
    
def plot_preds(**kwargs):
    # function to plot the log gauss predictions:
    data = kwargs.pop('data')
    plt.plot(data['xhat'], data['yhat'], **kwargs)

### BPN plot

In [ ]:
g = sns.FacetGrid(subdat, col='Observer', hue='flanked', hue_kws={'marker': ['o', 's']}, 
#                   col_wrap=2,
                  dropna=False,
                  size=2, legend_out=False)
g.map_dataframe(plot_preds, ls='-', ms=0)
g.map(log_error_wrap, 'log_freq', 'threshold', 'threshconfi_low', 'threshconfi_high',
      ls='')

# g.set(xlabel='Frequency (c/deg)', ylabel='Threshold')
g.set_xlabels('Frequency (c/deg)')
g.set_ylabels('Threshold')

x_labels = [1, 2, 4, 8, 16, 32]
x_ticks = np.log(x_labels)
g.set(xticks=x_ticks, xticklabels=x_labels)

y_labels = [.01, .02, .04, .08, .16]
y_ticks = np.log(y_labels)
g.set(yticks=y_ticks, yticklabels=y_labels)

# g.add_legend(title='')
g.fig.subplots_adjust(hspace=0.6, wspace=0.5)
sns.despine(trim=True, offset=5);
g.savefig(os.path.join(fig_dir, 'experiment_1_bpn.pdf'), bbox_inches='tight')

### Plot peak estimates

In [ ]:
param_dat

In [ ]:
param_dat.groupby(['flanked']).peak_x.mean()

In [ ]:
# difference in octaves:
np.log2(8.69 / 6.42)

In [ ]:
g = sns.stripplot(x='flanked', y='peak_x', data=param_dat,
                  jitter=0.05,
#               order=['unflanked', 'flanked'],
#               hue_order=['flanked', 'unflanked'],
                  palette=[pal[1], pal[0]])

g.set(xlabel='', ylabel='Peak frequency (c/deg)')
sns.despine(trim=True, offset=2)
g.set(yticks=[5, 7, 9, 11])
fig = plt.gcf()
fig.set_size_inches(3.5, 3)
plt.savefig(os.path.join(fig_dir, 'experiment_1_bpn_peaks.pdf'), 
            bbox_inches='tight',
            figsize=(1, 1))


### Export peak frequencies out for JASP analysis


In [ ]:
out = pd.pivot_table(param_dat, 
                     index=['Observer'], 
                     columns=['flanked'], 
                     values=['peak_x'])

# rename column names (from https://stackoverflow.com/questions/14507794/python-pandas-how-to-flatten-a-hierarchical-index-in-columns)
out.columns = ['_'.join(col).strip() for col in out.columns.values]
out

In [ ]:
# save to csv for analysis in JASP:
out.to_csv(os.path.join(top_dir, 'results', 'experiment_1', 'peak_estimates.csv'))

### RF plot

In [ ]:
# plot thresholds
subdat = dat.loc[dat['Distortion']=='RF', :]

g = sns.FacetGrid(subdat, col='Observer', hue='flanked',  hue_kws={'marker': ['o', 's']}, 
#                   col_wrap=2, 
                  size=2, legend_out=False)
g.map(log_error_wrap, 'log_freq', 'threshold', 'threshconfi_low', 'threshconfi_high',
      ls='')

# g.set(xlabel='Frequency (c/deg)', ylabel='Threshold')
g.set_xlabels('Frequency (c / $2\pi$)')
g.set_ylabels('Threshold')

x_labels = [1, 2, 4, 8, 16, 32]
x_ticks = np.log(x_labels)
g.set(xticks=x_ticks, xticklabels=x_labels)

y_labels = [.01, .02, .04, .1, .2, .4]
y_ticks = np.log(y_labels)
g.set(yticks=y_ticks, yticklabels=y_labels)

# plt.legend(title='', loc='lower right')
g.add_legend(title='')
g.fig.subplots_adjust(hspace=0.6, wspace=0.5)

sns.despine(trim=True, offset=5);
g.savefig(os.path.join(fig_dir, 'experiment_1_rf.pdf'), bbox_inches='tight')

### Performance as a function of target letter

In [ ]:
# read in the raw psi data:
fname = os.path.join(top_dir, 'results', 'experiment_1', 'all_data.csv')
# this file created by data_munging_expt_1.py
dat = pd.read_csv(fname)

# dat.loc[dat['subject']=='2', 'Observer'] = 'XXX'


dat.loc[dat['distortion']==' bex', 'Distortion Type'] = 'BPN'
dat.loc[dat['distortion']==' rf', 'Distortion Type'] = 'RF'

# remap subject numbers to initials:
dat.loc[dat['subject']==2, 'Observer'] = 'TW'
dat.loc[dat['subject']==5, 'Observer'] = 'ST'
dat.loc[dat['subject']==7, 'Observer'] = 'AM'
dat.loc[dat['subject']==8, 'Observer'] = 'RM'
dat.loc[dat['subject']==9, 'Observer'] = 'MF'

dat.info()

In [ ]:
dat.groupby(['targ_letter', 'distortion']).size()

In [ ]:
np.unique(dat['Distortion Type'])

In [ ]:
g = sns.factorplot('targ_letter', 'correct', hue='Distortion Type',
                   palette=pal, data=dat,
                   kind='point', size=3, linestyles=['', ''], legend_out=False)

g.set_xlabels('Target letter')
g.set_ylabels('Proportion correct')
g.set(yticks=[.5, .55, .6, .65, .7, .75])
sns.despine(trim=True, offset=0)
g.fig.set_figwidth(3)
g.savefig(os.path.join(fig_dir, 'experiment_1_targ_letter.pdf'), bbox_inches='tight')

In [ ]:
g = sns.factorplot('targ_letter', 'correct', hue='Distortion Type', col='Observer',
                   col_order=['AM', 'MF', 'RM', 'ST', 'TW'],
                   col_wrap=3,
                   palette=pal, data=dat,
                   kind='point', size=3, linestyles=['', ''], legend_out=True)

g.set_xlabels('Target letter')
g.set_ylabels('Proportion correct')
g.set(yticks=np.arange(.3, .9, step=.1))
sns.despine(trim=True, offset=0)
g.fig.subplots_adjust(hspace=0.5, wspace=0.3)
g.fig.set_figwidth(6)
g.fig.set_figheight(3.7)
g.savefig(os.path.join(fig_dir, 'experiment_1_targ_letter_by_subj.pdf'), bbox_inches='tight')

# Experiment 2

Note the labelling of the data files is "Experiment 3" here, because in the data collection we called each different distortion type (above) a separate experiment.

In [ ]:
pal = sns.color_palette(['#252525', '#636363', '#969696'])
# pal = sns.cubehelix_palette(3, start=1, rot=1.5, light=0.6, dark=0.3, reverse=True)
sns.set_palette(pal)
sns.palplot(pal)

In [ ]:
# check trial numbers by opening and grouping my munged file
# (produced by data_munging_expt_2.py)
fname = os.path.join(top_dir, 'results', 'experiment_2', 'all_data.csv')
dat = pd.read_csv(fname)
dat.info()

In [ ]:
dat.groupby(['experiment']).size()

### Prepare data

In [ ]:
fname = os.path.join(top_dir, 'results', 'saskia_analysis', 'sensitivitydata', 'alldatasensexp3c.csv')
dat = pd.read_csv(fname, sep='\t')
dat.info()

In [ ]:
dat.rename(columns=new_col, inplace=True)

dat.rename(columns={'subject': 'Observer',
                    'distortiontype': 'Distortion',
                    'distflanker': 'n_dist_flanks', 
                    'exp3': 'Experiment'}, 
           inplace=True)

dat.loc[dat['Distortion']==' Bex', 'Distortion'] = 'BPN'
dat.loc[dat['Distortion']==' RF', 'Distortion'] = 'RF'

dat.loc[dat['Experiment']==' a', 'Experiment'] = 'a'
dat.loc[dat['Experiment']==' b', 'Experiment'] = 'b'
dat.loc[dat['Experiment']==' c', 'Experiment'] = 'c'


In [ ]:
dat.info()

In [ ]:
dat

### BPN plot

In [ ]:
def log_error_wrap_dataframe(x, y, lo, hi, **kwargs):
    # needed because of hashtable error with unbalanced data.
    data = kwargs.pop('data')
    x = data[x].values
    x = x.astype(np.float)
    
    if np.any(data['Experiment']=='b'):
        x -= .3
    elif np.any(data['Experiment']=='c'):
        x += .3
        
    y = data[y].values
    lo = data[lo].values
    hi = data[hi].values
    err = np.array([np.log(y) - np.log(y - lo), np.log(y + hi) - np.log(y)])
    plt.errorbar(x, np.log(y), err, **kwargs)

In [ ]:
# plot thresholds
subdat = dat.loc[dat['Distortion']=='BPN', :]

g = sns.FacetGrid(subdat, col='Observer', 
                  hue='Experiment', 
                  hue_kws={'marker': ['o', 's', 'd']}, 
                  size=2, legend_out=True)
g.map_dataframe(log_error_wrap_dataframe, 'n_dist_flanks', 'threshold', 'threshconfi_low', 'threshconfi_high',
      ls='')

g.set(xlabel='No. distorted flankers', ylabel='Threshold')

x_labels = [0, 2, 4]
x_ticks = x_labels
g.set(xticks=x_ticks, xticklabels=x_labels)
g.set(xlim=(-0.2, 4.5))
y_labels = [.02, .04, .08, .16, .32]
y_ticks = np.log(y_labels)
g.set(yticks=y_ticks, yticklabels=y_labels)

g.add_legend()
g.fig.subplots_adjust(hspace=0.6, wspace=1)
sns.despine(trim=True, offset=5);
g.savefig(os.path.join(fig_dir, 'experiment_2_bpn.pdf'), bbox_inches='tight')

### RF plot

In [ ]:
# plot thresholds
subdat = dat.loc[dat['Distortion']=='RF', :]

g = sns.FacetGrid(subdat, col='Observer', 
                  hue='Experiment', 
                  hue_kws={'marker': ['o', 's', 'd']}, 
                  size=2, legend_out=True)
g.map_dataframe(log_error_wrap_dataframe, 'n_dist_flanks', 'threshold', 'threshconfi_low', 'threshconfi_high',
      ls='')

g.set(xlabel='No. distorted flankers', ylabel='Threshold')

x_labels = [0, 2, 4]
x_ticks = x_labels
g.set(xticks=x_ticks, xticklabels=x_labels)
g.set(xlim=(-0.2, 4.5))

y_labels = [.04, .1, .2, .4, .8]
y_ticks = np.log(y_labels)
g.set(yticks=y_ticks, yticklabels=y_labels)

g.add_legend()
g.fig.subplots_adjust(hspace=0.6, wspace=1)
sns.despine(trim=True, offset=5);
g.savefig(os.path.join(fig_dir, 'experiment_2_rf.pdf'), bbox_inches='tight')